In [1]:
import pandas

In [2]:
df = pandas.read_csv("metrics.csv", names=["basename", "seed", "i", "PSNR", "SSIM", "LPIPS"])

temp = df["basename"].str.split("_", n=2, expand=True)
temp[0] = temp[0] + " " + temp[1]
temp[2] = temp[2].replace("_", " ", regex=True).str.upper()
temp = temp[[0, 2]]

df[["task", "method"]] = temp
df = df.drop(["basename", "seed", "i"], axis="columns")

In [3]:
mean = df.groupby(["task", "method"]).mean().reset_index().round(2)
mean = mean.pivot(index="method", columns="task", values=["PSNR", "SSIM", "LPIPS"])
mean.swaplevel(0, 1, axis=1).sort_index(axis=1)

task         inpainting box              inpainting random               \
                      LPIPS   PSNR  SSIM             LPIPS   PSNR  SSIM   
method                                                                    
DIFFPIR 0010           0.97  14.23  0.13              0.78  10.97  0.32   
DIFFPIR 0100           0.88  15.03  0.20              0.68  10.26  0.25   
DIFFPIR 1000           0.90  14.65  0.18              0.74   9.51  0.21   
DPS 0010               0.64  10.41  0.34              0.58  12.68  0.43   
DPS 0100               0.38  16.82  0.50              0.39  16.66  0.49   
DPS 1000               0.22  21.02  0.64              0.19  21.90  0.66   
MMPS 0010 01           0.27  21.19  0.68              0.26  22.41  0.69   
MMPS 0010 05           0.23  21.73  0.69              0.20  23.72  0.75   
MMPS 0100 01           0.20  21.19  0.67              0.18  22.18  0.69   
MMPS 0100 05           0.20  21.30  0.67              0.15  22.82  0.72   
MMPS 1000 01           0.19  20.77  0.64              0.18  21.94  0.66   
MMPS 1000 05           0.20  20.98  0.64              0.14  22.52  0.69   
PGDM 0010              0.48  15.32  0.57              0.52  14.44  0.47   
PGDM 0100              0.92  14.18  0.22              0.23  21.59  0.66   
PGDM 1000              0.83  14.38  0.34              0.14  22.47  0.70   
TMPD 0010              0.36  19.90  0.64              0.59  11.08  0.40   
TMPD 0100              0.27  19.86  0.64              0.58  10.73  0.31   
TMPD 1000              0.25  19.53  0.62              0.68   9.98  0.25   

task         motion deblur              super resolution               
                     LPIPS   PSNR  SSIM            LPIPS   PSNR  SSIM  
method                                                                 
DIFFPIR 0010          0.39  23.98  0.51             0.31  26.45  0.69  
DIFFPIR 0100          0.17  25.17  0.71             0.19  26.30  0.73  
DIFFPIR 1000          0.17  24.67  0.68             0.19  25.67  0.68  
DPS 0010              0.75   8.63  0.27             0.58  12.01  0.41  
DPS 0100              0.29  19.75  0.57             0.35  18.29  0.54  
DPS 1000              0.18  22.91  0.66             0.16  25.02  0.72  
MMPS 0010 01          0.33  22.12  0.66             0.24  26.94  0.78  
MMPS 0010 05          0.20  26.70  0.78             0.18  28.02  0.81  
MMPS 0100 01          0.20  23.92  0.71             0.15  27.32  0.79  
MMPS 0100 05          0.13  26.70  0.77             0.13  27.12  0.78  
MMPS 1000 01          0.16  23.83  0.69             0.12  26.92  0.77  
MMPS 1000 05          0.11  26.18  0.75             0.11  26.60  0.76  
PGDM 0010             0.46  18.61  0.54             0.37  20.47  0.67  
PGDM 0100             0.84  17.02  0.34             0.45  21.80  0.51  
PGDM 1000             0.88  16.63  0.34             0.51  20.88  0.48  
TMPD 0010             0.27  25.28  0.74             0.26  26.07  0.76  
TMPD 0100             0.17  26.22  0.76             0.17  26.79  0.77  
TMPD 1000             0.14  25.91  0.74             0.14  26.53  0.76

In [4]:
rank = mean.rank(method="dense", ascending=False)
rank["LPIPS"] = mean["LPIPS"].rank(method="dense")

text = mean.map('{:.2f}'.format)
bold = r"\textbf{" + text + "}"
line = r"\underline{" + text + "}"

text = bold.where(rank == 1, text)
text = line.where(rank == 2, text)
text = text.swaplevel(0, 1, axis=1).sort_index(axis=1)

print(text.to_latex())

\begin{tabular}{lllllllllllll}
\toprule
task & \multicolumn{3}{r}{inpainting box} & \multicolumn{3}{r}{inpainting random} & \multicolumn{3}{r}{motion deblur} & \multicolumn{3}{r}{super resolution} \\
 & LPIPS & PSNR & SSIM & LPIPS & PSNR & SSIM & LPIPS & PSNR & SSIM & LPIPS & PSNR & SSIM \\
method &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
DIFFPIR 0010 & 0.97 & 14.23 & 0.13 & 0.78 & 10.97 & 0.32 & 0.39 & 23.98 & 0.51 & 0.31 & 26.45 & 0.69 \\
DIFFPIR 0100 & 0.88 & 15.03 & 0.20 & 0.68 & 10.26 & 0.25 & 0.17 & 25.17 & 0.71 & 0.19 & 26.30 & 0.73 \\
DIFFPIR 1000 & 0.90 & 14.65 & 0.18 & 0.74 & 9.51 & 0.21 & 0.17 & 24.67 & 0.68 & 0.19 & 25.67 & 0.68 \\
DPS 0010 & 0.64 & 10.41 & 0.34 & 0.58 & 12.68 & 0.43 & 0.75 & 8.63 & 0.27 & 0.58 & 12.01 & 0.41 \\
DPS 0100 & 0.38 & 16.82 & 0.50 & 0.39 & 16.66 & 0.49 & 0.29 & 19.75 & 0.57 & 0.35 & 18.29 & 0.54 \\
DPS 1000 & 0.22 & 21.02 & 0.64 & 0.19 & 21.90 & 0.66 & 0.18 & 22.91 & 0.66 & 0.16 & 25.02 & 0.72 \\
MMPS 0010 01 & 0.27 & 21.19 & \underline{0.